In [1]:
import io
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)

In [2]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Functions.get_previous_date(date_tday)
date_bf_yday = Functions.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 11, 3),
 datetime.date(2020, 11, 2),
 datetime.date(2020, 11, 1))

In [3]:
# Today url for regions file
url_tday = Functions.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Functions.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Functions.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201103.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201102.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201101.csv')

In [4]:
wait = True
if wait:
    while True:
        response = requests.get(url_tday)
        if response.status_code == 200:
            print('Presenti dati odierni')
            date_last = date_tday
            csv = response.content
            df_raw_r_1 = pd.read_csv(io.StringIO(csv.decode('utf-8')))
            df_raw_r_0 = pd.read_csv(url_yday)
            break
        else:
            print('Status code: {}. Retrying...'.format(status_code))
        time.sleep(20)
else:
    print('Dati di ieri')
    date_last = date_yday
    df_raw_r_1 = pd.read_csv(url_yday)
    df_raw_r_0 = pd.read_csv(url_bf_yday)

Presenti dati odierni


In [5]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-03T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3379,213,3592,35121,...,2618,3169,34675,4444,35311,42521,77832,1063115,664753,NaN
1,2020-11-03T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,170,13,183,1867,...,38,94,1361,180,3164,427,3591,42561,26492,NaN
2,2020-11-03T17:00:00,ITA,3,Lombardia,45.466794,9.190347,4740,475,5215,93351,...,4148,6804,100115,17752,171744,44689,216433,3040698,1889116,NaN
3,2020-11-03T17:00:00,ITA,5,Veneto,45.434905,12.338452,1011,142,1153,34013,...,2233,2298,25471,2458,25215,37880,63095,2362039,917684,NaN
4,2020-11-03T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,205,39,244,5514,...,205,366,5659,411,10476,1352,11828,550607,233148,NaN
5,2020-11-03T17:00:00,ITA,7,Liguria,44.411493,8.932699,1147,64,1211,8618,...,535,1052,19356,1810,21800,9195,30995,457185,235736,NaN
6,2020-11-03T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1464,153,1617,26731,...,1856,1912,28123,4687,44063,17095,61158,1635729,874120,In seguito a verifica sui dati sono stati elim...
7,2020-11-03T17:00:00,ITA,9,Toscana,43.769231,11.255889,1261,190,1451,32466,...,2182,2336,15667,1403,41133,9854,50987,1133147,753032,effettuati 15775 tamponi rapidi antigenici di ...
8,2020-11-03T17:00:00,ITA,10,Umbria,43.106758,12.388247,305,48,353,7214,...,241,420,3845,148,3360,8200,11560,307285,178056,NaN
9,2020-11-03T17:00:00,ITA,11,Marche,43.616760,13.518875,374,53,427,6990,...,375,431,7160,1031,15421,187,15608,328103,192435,NaN


In [6]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-02T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3108,196,3304,32791,...,1681,2003,34153,4415,34275,40388,74663,1050421,652762,NaN
1,2020-11-02T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,167,12,179,1833,...,92,122,1306,179,3080,417,3497,42151,26356,NaN
2,2020-11-02T17:00:00,ITA,3,Lombardia,45.466794,9.190347,4406,435,4841,89577,...,4343,5278,97576,17635,166581,43048,209629,3008361,1871538,NaN
3,2020-11-02T17:00:00,ITA,5,Veneto,45.434905,12.338452,873,136,1009,31924,...,1519,1544,25437,2427,25103,35694,60797,2350517,912972,NaN
4,2020-11-02T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,179,37,216,5337,...,177,218,5506,403,10146,1316,11462,545374,231668,NaN
5,2020-11-02T17:00:00,ITA,7,Liguria,44.411493,8.932699,1133,57,1190,8104,...,580,599,18859,1790,21510,8433,29943,450566,232521,NaN
6,2020-11-02T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1382,138,1520,24972,...,1575,1652,28093,4664,43112,16137,59249,1612029,865327,NaN
7,2020-11-02T17:00:00,ITA,9,Toscana,43.769231,11.255889,1217,182,1399,30336,...,1761,2009,15529,1387,39186,9465,48651,1118732,743951,effettuati 15426 tamponi rapidi antigenici di ...
8,2020-11-02T17:00:00,ITA,10,Umbria,43.106758,12.388247,308,46,354,6972,...,116,193,3674,140,3290,7850,11140,302745,175793,NaN
9,2020-11-02T17:00:00,ITA,11,Marche,43.616760,13.518875,334,50,384,6658,...,319,373,7110,1025,15000,177,15177,325563,190758,NaN


In [7]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-03T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3379,213,3592,35121,...,2618,3169,34675,4444,35311,42521,77832,1063115,664753,NaN
1,2020-11-03T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,170,13,183,1867,...,38,94,1361,180,3164,427,3591,42561,26492,NaN
2,2020-11-03T17:00:00,ITA,3,Lombardia,45.466794,9.190347,4740,475,5215,93351,...,4148,6804,100115,17752,171744,44689,216433,3040698,1889116,NaN
3,2020-11-03T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,487,40,527,8287,...,585,607,9914,767,15299,4196,19495,535256,239050,NaN
4,2020-11-03T17:00:00,ITA,5,Veneto,45.434905,12.338452,1011,142,1153,34013,...,2233,2298,25471,2458,25215,37880,63095,2362039,917684,NaN
5,2020-11-03T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,205,39,244,5514,...,205,366,5659,411,10476,1352,11828,550607,233148,NaN
6,2020-11-03T17:00:00,ITA,7,Liguria,44.411493,8.932699,1147,64,1211,8618,...,535,1052,19356,1810,21800,9195,30995,457185,235736,NaN
7,2020-11-03T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1464,153,1617,26731,...,1856,1912,28123,4687,44063,17095,61158,1635729,874120,In seguito a verifica sui dati sono stati elim...
8,2020-11-03T17:00:00,ITA,9,Toscana,43.769231,11.255889,1261,190,1451,32466,...,2182,2336,15667,1403,41133,9854,50987,1133147,753032,effettuati 15775 tamponi rapidi antigenici di ...
9,2020-11-03T17:00:00,ITA,10,Umbria,43.106758,12.388247,305,48,353,7214,...,241,420,3845,148,3360,8200,11560,307285,178056,NaN


In [8]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-11-02T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3108,196,3304,...,1681,2003,34153,4415,34275,40388,74663,1050421,652762,NaN
1,17,2020-11-02T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,167,12,179,...,92,122,1306,179,3080,417,3497,42151,26356,NaN
2,8,2020-11-02T17:00:00,ITA,3,Lombardia,45.466794,9.190347,4406,435,4841,...,4343,5278,97576,17635,166581,43048,209629,3008361,1871538,NaN
3,19,2020-11-02T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,471,34,505,...,498,624,9896,763,14768,4120,18888,531143,237796,NaN
4,18,2020-11-02T17:00:00,ITA,5,Veneto,45.434905,12.338452,873,136,1009,...,1519,1544,25437,2427,25103,35694,60797,2350517,912972,NaN
5,5,2020-11-02T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,179,37,216,...,177,218,5506,403,10146,1316,11462,545374,231668,NaN
6,7,2020-11-02T17:00:00,ITA,7,Liguria,44.411493,8.932699,1133,57,1190,...,580,599,18859,1790,21510,8433,29943,450566,232521,NaN
7,4,2020-11-02T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1382,138,1520,...,1575,1652,28093,4664,43112,16137,59249,1612029,865327,NaN
8,15,2020-11-02T17:00:00,ITA,9,Toscana,43.769231,11.255889,1217,182,1399,...,1761,2009,15529,1387,39186,9465,48651,1118732,743951,effettuati 15426 tamponi rapidi antigenici di ...
9,16,2020-11-02T17:00:00,ITA,10,Umbria,43.106758,12.388247,308,46,354,...,116,193,3674,140,3290,7850,11140,302745,175793,NaN


In [9]:
# Print raw columns
# print(df_r_1.columns, df_r_0.columns)
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rf_1 = df_r_1.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0 = df_r_0.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
# df_rf_1.columns, df_rf_0.columns

In [10]:
df_rf_0 = df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0;

In [11]:
df_r = df_rf_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-03T17:00:00,1,Piemonte,3379,213,3592,35121,38713,2618,3169,34675,4444,35311,42521,77832,1063115,664753,4341375
1,2020-11-03T17:00:00,2,Valle d'Aosta,170,13,183,1867,2050,38,94,1361,180,3164,427,3591,42561,26492,125501
2,2020-11-03T17:00:00,3,Lombardia,4740,475,5215,93351,98566,4148,6804,100115,17752,171744,44689,216433,3040698,1889116,10103969
3,2020-11-03T17:00:00,4,Trentino-Alto Adige,487,40,527,8287,8814,585,607,9914,767,15299,4196,19495,535256,239050,1074819
4,2020-11-03T17:00:00,5,Veneto,1011,142,1153,34013,35166,2233,2298,25471,2458,25215,37880,63095,2362039,917684,4907704
5,2020-11-03T17:00:00,6,Friuli Venezia Giulia,205,39,244,5514,5758,205,366,5659,411,10476,1352,11828,550607,233148,1211357
6,2020-11-03T17:00:00,7,Liguria,1147,64,1211,8618,9829,535,1052,19356,1810,21800,9195,30995,457185,235736,1543127
7,2020-11-03T17:00:00,8,Emilia-Romagna,1464,153,1617,26731,28348,1856,1912,28123,4687,44063,17095,61158,1635729,874120,4467118
8,2020-11-03T17:00:00,9,Toscana,1261,190,1451,32466,33917,2182,2336,15667,1403,41133,9854,50987,1133147,753032,3722729
9,2020-11-03T17:00:00,10,Umbria,305,48,353,7214,7567,241,420,3845,148,3360,8200,11560,307285,178056,880285


In [12]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-03T17:00:00,1,Piemonte,3379,213,3592,35121,38713,2618,3169,34675,4444,35311,42521,77832,1063115,664753,4341375,367
1,2020-11-03T17:00:00,2,Valle d'Aosta,170,13,183,1867,2050,38,94,1361,180,3164,427,3591,42561,26492,125501,20
2,2020-11-03T17:00:00,3,Lombardia,4740,475,5215,93351,98566,4148,6804,100115,17752,171744,44689,216433,3040698,1889116,10103969,983
3,2020-11-03T17:00:00,4,Trentino-Alto Adige,487,40,527,8287,8814,585,607,9914,767,15299,4196,19495,535256,239050,1074819,106
4,2020-11-03T17:00:00,5,Veneto,1011,142,1153,34013,35166,2233,2298,25471,2458,25215,37880,63095,2362039,917684,4907704,825
5,2020-11-03T17:00:00,6,Friuli Venezia Giulia,205,39,244,5514,5758,205,366,5659,411,10476,1352,11828,550607,233148,1211357,175
6,2020-11-03T17:00:00,7,Liguria,1147,64,1211,8618,9829,535,1052,19356,1810,21800,9195,30995,457185,235736,1543127,209
7,2020-11-03T17:00:00,8,Emilia-Romagna,1464,153,1617,26731,28348,1856,1912,28123,4687,44063,17095,61158,1635729,874120,4467118,516
8,2020-11-03T17:00:00,9,Toscana,1261,190,1451,32466,33917,2182,2336,15667,1403,41133,9854,50987,1133147,753032,3722729,415
9,2020-11-03T17:00:00,10,Umbria,305,48,353,7214,7567,241,420,3845,148,3360,8200,11560,307285,178056,880285,70


In [13]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [14]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-03T17:00:00,3,Lombardia,4740,475,5215,93351,98566,4148,6804,...,48.32,3774,1526,32337,17578,117,2539,21.04,0.06734,2.14
1,2020-11-03T17:00:00,1,Piemonte,3379,213,3592,35121,38713,2618,3169,...,58.04,2330,1166,12694,11991,29,522,24.96,0.07300,1.79
2,2020-11-03T17:00:00,15,Campania,1497,227,1724,48995,50719,1949,2971,...,53.16,1882,110,13801,8469,24,998,21.53,0.05135,1.13
3,2020-11-03T17:00:00,9,Toscana,1261,190,1451,32466,33917,2182,2336,...,45.78,2130,327,14415,9081,16,138,16.21,0.06275,1.37
4,2020-11-03T17:00:00,5,Veneto,1011,142,1153,34013,35166,2233,2298,...,17.21,2089,754,11522,4712,31,34,19.94,0.04682,1.29
5,2020-11-03T17:00:00,12,Lazio,2192,197,2389,37340,39729,1946,2209,...,26.37,1774,350,25481,19915,21,242,8.67,0.03766,0.90
6,2020-11-03T17:00:00,8,Emilia-Romagna,1464,153,1617,26731,28348,1856,1912,...,29.65,1759,260,23700,8793,23,30,8.07,0.04280,1.37
7,2020-11-03T17:00:00,16,Puglia,749,95,844,12681,13525,956,1163,...,25.96,905,537,5955,6224,13,194,19.53,0.02901,0.53
8,2020-11-03T17:00:00,7,Liguria,1147,64,1211,8618,9829,535,1052,...,30.62,514,453,6619,3215,20,497,15.89,0.06817,2.01
9,2020-11-03T17:00:00,19,Sicilia,1072,150,1222,15584,16806,742,1048,...,27.88,687,24,8015,4356,14,292,13.08,0.02109,0.50


In [15]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)